In [1]:
import requests
import json
import pandas as pd
from pathlib import Path
import yfinance as yf
from pytrends.request import TrendReq
from datetime import datetime, timedelta

from dotenv import load_dotenv
import os

# Cargar variables desde el archivo .env
load_dotenv()

True

In [ ]:
api_key = os.getenv("ALPHAVANTAGE_API_KEY")
symbol = 'BTC'
market = 'USD'
url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey={api_key}'

# Obtener los datos
response = requests.get(url)
data = response.json()

# Guardar JSON en archivo
with open('btc_daily.json', 'w') as f:
    json.dump(data, f, indent=4)

# Extraer la serie temporal
time_series = data.get("Time Series (Digital Currency Daily)", {})

# Convertir a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')
df.index = pd.to_datetime(df.index)
df = df.sort_index()

In [3]:
# Renombrar columnas para facilidad de uso
df.columns = [
    "open_usd",
    "high_usd",
    "low_usd",
    "close_usd",
    "volume"
]

# Convertir a valores numéricos
cols_to_numeric = ["open_usd", "high_usd", "low_usd", "close_usd", "volume"]
df[cols_to_numeric] = df[cols_to_numeric].astype(float)

# Mostrar primeras filas
print(df.head())

            open_usd  high_usd   low_usd  close_usd        volume
2024-06-25  60262.16  62400.00  60217.36   61789.71  15201.582477
2024-06-26  61794.47  62470.00  60656.80   60816.68  11392.798350
2024-06-27  60818.86  62346.16  60546.94   61615.39  10530.358988
2024-06-28  61611.43  62170.62  59868.00   60313.35  11381.025874
2024-06-29  60312.36  61122.66  60273.80   60885.67   3199.036501


In [4]:
symbol = 'BTC'
market = 'USD'
url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&outputsize=full&apikey={api_key}'

# Carpeta de salida
output_dir = Path("btc_data_by_year")
output_dir.mkdir(exist_ok=True)

# Obtener los datos
response = requests.get(url)
data = response.json()

# Guardar el JSON completo por si se necesita
with open(output_dir / 'btc_full.json', 'w') as f:
    json.dump(data, f, indent=4)

# Procesar la serie temporal
time_series = data.get("Time Series (Digital Currency Daily)", {})
df = pd.DataFrame.from_dict(time_series, orient='index')
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# Renombrar columnas según estructura simple
df.columns = ["open", "high", "low", "close", "volume"]
df = df.astype(float)


TypeError: keys must be str, int, float, bool or None, not Timestamp

In [5]:
# Guardar por año
for year, group in df.groupby(df.index.year):
    group.index = group.index.strftime("%Y-%m-%d")
    group_dict = group.sort_index(ascending=False).to_dict(orient='index')
    with open(output_dir / f'btc_{year}.json', 'w') as f:
        json.dump(group_dict, f, indent=4)

print(f"✔ Datos completos guardados por año en: {output_dir.resolve()}")

✔ Datos completos guardados por año en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/notebooks/btc_data_by_year


In [1]:
# Descargar histórico completo de BTC/USD desde 2010
btc = yf.download("BTC-USD", start="2010-01-01")

# Guardar como CSV por año
for year, group in btc.groupby(btc.index.year):
    group.to_csv(f"btc_{year}.csv")

print("✔ Histórico completo guardado como CSV por año.")


/var/folders/c_/rq9m5qb17jsg0xbdp8j28n2c0000gn/T/ipykernel_78804/936364403.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc = yf.download("BTC-USD", start="2010-01-01")
[*********************100%***********************]  1 of 1 completed

✔ Histórico completo guardado como CSV por año.


In [5]:
from pathlib import Path

# Lista de activos: (subcarpeta, símbolo_yfinance)
assets = [
    ("sp500", "^GSPC"),
    ("gold", "GC=F"),
    ("eth", "ETH-USD")
]

# Rango de fechas
start_date = "2010-01-01"

for name, symbol in assets:
    print(f"⬇ Descargando datos de {name.upper()} ({symbol})...")
    data = yf.download(symbol, start=start_date, auto_adjust=True)

    # Crear carpeta de salida: dataset/{activo}
    output_dir = Path("../datasets_old") / name
    output_dir.mkdir(parents=True, exist_ok=True)

    # Guardar por año
    for year, group in data.groupby(data.index.year):
        filename = output_dir / f"{name}_{year}.csv"
        group.to_csv(filename)

    print(f"✔ Datos de {name.upper()} guardados en: {output_dir.resolve()}\n")


⬇ Descargando datos de SP500 (^GSPC)...


[*********************100%***********************]  1 of 1 completed


✔ Datos de SP500 guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/sp500

⬇ Descargando datos de GOLD (GC=F)...


[*********************100%***********************]  1 of 1 completed


✔ Datos de GOLD guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/gold

⬇ Descargando datos de ETH (ETH-USD)...


[*********************100%***********************]  1 of 1 completed

✔ Datos de ETH guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/eth



In [2]:
from pathlib import Path

# Lista de activos: (subcarpeta, símbolo_yfinance)
assets = [
    ("btc2", "BTC-USD")
]

# Rango de fechas
start_date = "2010-01-01"

for name, symbol in assets:
    print(f"⬇ Descargando datos de {name.upper()} ({symbol})...")
    data = yf.download(symbol, start=start_date, auto_adjust=True)

    # Crear carpeta de salida: dataset/{activo}
    output_dir = Path("../datasets_old") / name
    output_dir.mkdir(parents=True, exist_ok=True)

    # Guardar por año
    for year, group in data.groupby(data.index.year):
        filename = output_dir / f"{name}_{year}.csv"
        group.to_csv(filename)

    print(f"✔ Datos de {name.upper()} guardados en: {output_dir.resolve()}\n")


⬇ Descargando datos de BTC2 (BTC-USD)...


[*********************100%***********************]  1 of 1 completed

✔ Datos de BTC2 guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/btc2



In [6]:
# DXY: Dollar Index
symbol = "DX-Y.NYB"
name = "dxy"
start_date = "2010-01-01"

print(f"⬇ Descargando datos de DOLLAR INDEX ({symbol})...")
data = yf.download(symbol, start=start_date, auto_adjust=True)

# Crear carpeta de salida: dataset/dxy
output_dir = Path("../datasets_old") / name
output_dir.mkdir(parents=True, exist_ok=True)

# Guardar por año
for year, group in data.groupby(data.index.year):
    filename = output_dir / f"{name}_{year}.csv"
    group.to_csv(filename)

print(f"✔ Datos del DOLLAR INDEX guardados en: {output_dir.resolve()}")

⬇ Descargando datos de DOLLAR INDEX (DX-Y.NYB)...


[*********************100%***********************]  1 of 1 completed

✔ Datos del DOLLAR INDEX guardados en: /Users/cbarril/dev/posgrado/tp_ast1_19co2024/datasets/dxy


In [8]:
pytrends = TrendReq(hl='en-US', tz=0)

# Parámetros
kw_list = ["bitcoin"]
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 6, 9)
delta = timedelta(days=270)  # máximo rango diario permitido por Google Trends

current_start = start_date

while current_start < end_date:
    current_end = min(current_start + delta, end_date)
    timeframe = current_start.strftime('%Y-%m-%d') + " " + current_end.strftime('%Y-%m-%d')

    # Cargar datos
    try:
        pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='', gprop='')
        data = pytrends.interest_over_time()

        if data.empty:
            print(f"No data for range: {timeframe}")
        else:
            if 'isPartial' in data.columns:
                data = data.drop(columns='isPartial')

            # Obtener el año del inicio del tramo
            year = current_start.year
            output_dir = f"../datasets_old/trend/{year}"
            os.makedirs(output_dir, exist_ok=True)

            filename = f"{output_dir}/bitcoin_trend_{current_start.strftime('%Y-%m-%d')}_to_{current_end.strftime('%Y-%m-%d')}.csv"
            data.to_csv(filename)
            print(f"Saved: {filename}")

    except Exception as e:
        print(f"Error with range {timeframe}: {e}")

    current_start = current_end  # avanzar al siguiente tramo


Saved: ../datasets/trend/2025/bitcoin_trend_2025-01-01_to_2025-06-09.csv


/Users/cbarril/dev/posgrado/tp_ast1_19co2024/ast1/lib/python3.12/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [11]:
import requests

url = "https://api.alternative.me/fng/?limit=0&format=json"
response = requests.get(url)
data = response.json()["data"]

# Convertir a DataFrame
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
df['value'] = df['value'].astype(int)
df = df.sort_values('timestamp')

# Ver columnas disponibles
print(df.head())

# Guardar
df.to_csv("fear_and_greed_index.csv", index=False)

      value value_classification  timestamp time_until_update
2681     30                 Fear 2018-02-01               NaN
2680     15         Extreme Fear 2018-02-02               NaN
2679     40                 Fear 2018-02-03               NaN
2678     24         Extreme Fear 2018-02-04               NaN
2677     11         Extreme Fear 2018-02-05               NaN


In [12]:
import pandas as pd
import os

# Descargar datos desde la API de Blockchain.com
url = "https://api.blockchain.info/charts/n-unique-addresses?timespan=all&format=csv"
df = pd.read_csv(url)
df.columns = ['date', 'active_addresses']
df['date'] = pd.to_datetime(df['date'])

# Crear carpeta base si no existe
base_path = "../datasets_old/active_addresses"
os.makedirs(base_path, exist_ok=True)

# Separar y guardar por año
for year, group in df.groupby(df['date'].dt.year):
    year_path = os.path.join(base_path, f"{year}.csv")
    group.to_csv(year_path, index=False)
    print(f"Guardado: {year_path}")


Guardado: ../datasets/active_addresses/2009.csv
Guardado: ../datasets/active_addresses/2010.csv
Guardado: ../datasets/active_addresses/2011.csv
Guardado: ../datasets/active_addresses/2012.csv
Guardado: ../datasets/active_addresses/2013.csv
Guardado: ../datasets/active_addresses/2014.csv
Guardado: ../datasets/active_addresses/2015.csv
Guardado: ../datasets/active_addresses/2016.csv
Guardado: ../datasets/active_addresses/2017.csv
Guardado: ../datasets/active_addresses/2018.csv
Guardado: ../datasets/active_addresses/2019.csv
Guardado: ../datasets/active_addresses/2020.csv
Guardado: ../datasets/active_addresses/2021.csv
Guardado: ../datasets/active_addresses/2022.csv
Guardado: ../datasets/active_addresses/2023.csv
Guardado: ../datasets/active_addresses/2024.csv
Guardado: ../datasets/active_addresses/2025.csv


In [13]:
import pandas as pd
import os
import requests

fed_api_key = os.getenv("FED_API_KEY")

# Endpoint para tasa de interés FED
series_id = "FEDFUNDS"
url = f"https://api.stlouisfed.org/fred/series/observations"
params = {
    "series_id": series_id,
    "api_key": fed_api_key,
    "file_type": "json",
}

response = requests.get(url, params=params)
data = response.json()["observations"]

# Crear DataFrame
df = pd.DataFrame(data)[["date", "value"]]
df.columns = ["date", "interest_rate"]
df["date"] = pd.to_datetime(df["date"])
df["interest_rate"] = pd.to_numeric(df["interest_rate"], errors="coerce")

# Crear carpeta base
base_path = "../datasets_old/interest_rate"
os.makedirs(base_path, exist_ok=True)

# Guardar CSV por año
for year, group in df.groupby(df['date'].dt.year):
    year_path = os.path.join(base_path, f"{year}.csv")
    group.to_csv(year_path, index=False)
    print(f"Guardado: {year_path}")


Guardado: ../datasets/interest_rate/1954.csv
Guardado: ../datasets/interest_rate/1955.csv
Guardado: ../datasets/interest_rate/1956.csv
Guardado: ../datasets/interest_rate/1957.csv
Guardado: ../datasets/interest_rate/1958.csv
Guardado: ../datasets/interest_rate/1959.csv
Guardado: ../datasets/interest_rate/1960.csv
Guardado: ../datasets/interest_rate/1961.csv
Guardado: ../datasets/interest_rate/1962.csv
Guardado: ../datasets/interest_rate/1963.csv
Guardado: ../datasets/interest_rate/1964.csv
Guardado: ../datasets/interest_rate/1965.csv
Guardado: ../datasets/interest_rate/1966.csv
Guardado: ../datasets/interest_rate/1967.csv
Guardado: ../datasets/interest_rate/1968.csv
Guardado: ../datasets/interest_rate/1969.csv
Guardado: ../datasets/interest_rate/1970.csv
Guardado: ../datasets/interest_rate/1971.csv
Guardado: ../datasets/interest_rate/1972.csv
Guardado: ../datasets/interest_rate/1973.csv
Guardado: ../datasets/interest_rate/1974.csv
Guardado: ../datasets/interest_rate/1975.csv
Guardado: 